In [ ]:
import pandas as pd
import requests
from pathlib import Path
from datetime import datetime

BASE_URL = "https://chilamboli.devmorphix.com"

CATEGORY_TO_LETTER = {
    "Sub Junior": "B", "Junior": "J", "Senior": "S", "Combined": "C", "Special": "P"
}

# Setup output directory
notebooks_dir = Path.cwd() if Path.cwd().name == 'notebooks' else Path.cwd() / 'notebooks'
output_dir = notebooks_dir / 'output'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Config loaded: BASE_URL={BASE_URL}")
print(f"Category mappings: {len(CATEGORY_TO_LETTER)} categories")
print(f"Output directory: {output_dir}")

In [ ]:
# Track event letters within each category (A, B, C, ...)
_event_letters = {}  # Maps (category, event_name) -> letter
_category_event_counters = {}  # Tracks how many events per category
_chest_counters = {}  # Maps (category, event_letter) -> counter (starts from 101)

def get_category_letter(age_category):
    """Get the category letter (S, J, B, C, P)"""
    return CATEGORY_TO_LETTER.get(age_category.strip())

def get_event_letter(category_letter, event_name):
    """Get or assign a letter (A, B, C, ...) for an event within a category"""
    key = (category_letter, event_name.strip())
    
    if key not in _event_letters:
        # Assign next available letter for this category
        category_count = _category_event_counters.get(category_letter, 0)
        event_letter = chr(ord('A') + category_count)  # A, B, C, ...
        _event_letters[key] = event_letter
        _category_event_counters[category_letter] = category_count + 1
    
    return _event_letters[key]

def next_chest_number(event_name, age_category):
    """Generate next chest number: {category}{event_letter}{number}
    Format: SA101, SA102, ..., SB101, SB102, etc.
    Numbers start from 101"""
    cat = get_category_letter(age_category)
    if not cat:
        return None
    
    event_letter = get_event_letter(cat, event_name)
    counter_key = (cat, event_letter)
    
    # Initialize counter at 101 if first time
    if counter_key not in _chest_counters:
        _chest_counters[counter_key] = 101
    else:
        _chest_counters[counter_key] += 1
    
    number = _chest_counters[counter_key]
    chest_num = f"{cat}{event_letter}{number}"
    return chest_num, chest_num

def reset_counters():
    """Reset all counters and mappings"""
    _event_letters.clear()
    _category_event_counters.clear()
    _chest_counters.clear()

def _fetch_paginated(path, limit=100, **params):
    """Fetch all data from paginated API endpoint"""
    page, all_data = 1, []
    while True:
        r = requests.get(f"{BASE_URL}{path}", params={**params, "page": page, "limit": limit}, timeout=30)
        r.raise_for_status()
        j = r.json()
        # Handle both response formats: with/without "success" field
        if "success" in j and not j.get("success"):
            raise RuntimeError(j.get("message", "API error"))
        data = j.get("data", [])
        all_data.extend(data)
        meta = j.get("metadata", {})
        total = meta.get("total", 0)
        if len(all_data) >= total or not data:
            break
        page += 1
    return all_data

print("Helpers defined: get_category_letter, get_event_letter, next_chest_number, reset_counters, _fetch_paginated")

In [ ]:
# Generate Excel sheet with all registrations and chest numbers
print("Fetching all registrations...")
all_regs = _fetch_paginated("/api/registrations", limit=100)
print(f"  -> fetched {len(all_regs)} registration(s)")

# Fetch schools for school names
print("Fetching schools...")
schools_data = _fetch_paginated("/api/schools", limit=100)
school_map = {s["id"]: s.get("name", "Unknown") for s in schools_data}
print(f"  -> fetched {len(schools_data)} school(s)")

# Generate chest numbers for all registrations
print("\nGenerating chest numbers...")
reset_counters()
excel_data = []

for reg in all_regs:
    event = reg.get("event", {})
    event_name = event.get("name", "")
    age_category = event.get("ageCategory", "")
    
    gen = next_chest_number(event_name, age_category)
    if not gen:
        print(f"[skip] no category mapping: {event_name!r} / {age_category!r}")
        continue
    
    chest_num, disp = gen
    school_name = school_map.get(reg.get("schoolId", ""), "Unknown")
    
    excel_data.append({
        "Chest Number": disp,
        "Event Name": event_name,
        "Age Category": age_category,
        "Team Name": reg.get("teamName", ""),
        "School Name": school_name,
    })

print(f"  -> generated {len(excel_data)} chest number(s)")

# Show event letter assignments
print("\nEvent letter assignments by category:")
for (cat, event_name), letter in sorted(_event_letters.items()):
    print(f"  {cat}{letter}: {event_name}")

# Create DataFrame and export to Excel
df = pd.DataFrame(excel_data)
df = df.sort_values(["Age Category", "Event Name", "Chest Number"])

# Formatting functions (similar to data_processing.ipynb)
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

def format_worksheet(worksheet, df):
    """Apply formatting to Excel worksheet"""
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)
    
    # Format header row
    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    
    # Auto-size columns
    for idx, column in enumerate(df.columns, 1):
        column_letter = get_column_letter(idx)
        max_length = max(len(str(column)), df[column].astype(str).map(len).max() if len(df) > 0 else 0)
        worksheet.column_dimensions[column_letter].width = min(max(max_length + 2, 10), 50)
    
    # Freeze header row
    worksheet.freeze_panes = "A2"
    worksheet.row_dimensions[1].height = 25

# Generate Excel file with formatting
output_file = output_dir / f'chest_numbers_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'

try:
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Chest Numbers', index=False)
        format_worksheet(writer.sheets['Chest Numbers'], df)
    
    print(f"\n✓ Excel file generated: {output_file}")
    print(f"  Total rows: {len(df)}")
    print(f"  Columns: {', '.join(df.columns)}")
except ImportError:
    print("\n⚠️  openpyxl not installed. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Chest Numbers', index=False)
        format_worksheet(writer.sheets['Chest Numbers'], df)
    
    print(f"\n✓ Excel file generated: {output_file}")
    print(f"  Total rows: {len(df)}")
    print(f"  Columns: {', '.join(df.columns)}")